In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
pd.set_option('display.max_columns', None)  

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

In [2]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML, get_leaderboard
from sklearn.model_selection import StratifiedShuffleSplit

In [3]:
df = pd.read_csv('insurance_claims.csv')

print(df.shape)
df.head()

(1000, 40)


,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,fraud_reported,_c39
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,2015-01-25,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,Y,NaN
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,2015-01-21,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,Y,NaN
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,N,NaN
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,2015-01-10,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,Y,NaN
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,2015-02-17,Vehicle Theft,?,Minor Damage,None,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,N,NaN


Data overview

In [4]:
df.isnull().sum()

months_as_customer                0
age                               0
policy_number                     0
policy_bind_date                  0
policy_state                      0
policy_csl                        0
policy_deductable                 0
policy_annual_premium             0
umbrella_limit                    0
insured_zip                       0
insured_sex                       0
insured_education_level           0
insured_occupation                0
insured_hobbies                   0
insured_relationship              0
capital-gains                     0
capital-loss                      0
incident_date                     0
incident_type                     0
collision_type                    0
incident_severity                 0
authorities_contacted             0
incident_state                    0
incident_city                     0
incident_location                 0
incident_hour_of_the_day          0
number_of_vehicles_involved       0
property_damage             

In [5]:
# drop column _c39 because it's null

df = df.drop(['_c39'], axis = 1)

In [6]:
df.dtypes

months_as_customer               int64
age                              int64
policy_number                    int64
policy_bind_date                object
policy_state                    object
policy_csl                      object
policy_deductable                int64
policy_annual_premium          float64
umbrella_limit                   int64
insured_zip                      int64
insured_sex                     object
insured_education_level         object
insured_occupation              object
insured_hobbies                 object
insured_relationship            object
capital-gains                    int64
capital-loss                     int64
incident_date                   object
incident_type                   object
collision_type                  object
incident_severity               object
authorities_contacted           object
incident_state                  object
incident_city                   object
incident_location               object
incident_hour_of_the_day 

Prepare data

In [7]:
# Set policy_bind_date type as date
df['policy_bind_date'] = pd.to_datetime(df['policy_bind_date'])
df['incident_date'] = pd.to_datetime(df['incident_date'])

In [8]:
# How long before accident
df['incident_date2policy_bind_date'] = (df['incident_date'] - df['policy_bind_date']).dt.days

In [9]:
# set the target
target_cols = pd.get_dummies(df['fraud_reported'], drop_first=True).astype(str)
target_cols.columns = ['target']

In [11]:
df.dtypes

months_as_customer                         int64
age                                        int64
policy_number                              int64
policy_bind_date                  datetime64[ns]
policy_state                              object
policy_csl                                object
policy_deductable                          int64
policy_annual_premium                    float64
umbrella_limit                             int64
insured_zip                                int64
insured_sex                               object
insured_education_level                   object
insured_occupation                        object
insured_hobbies                           object
insured_relationship                      object
capital-gains                              int64
capital-loss                               int64
incident_date                     datetime64[ns]
incident_type                             object
collision_type                            object
incident_severity   

In [12]:
dummies_df=[target_cols]
df = pd.concat([df] + dummies_df, axis=1)

df.drop(['fraud_reported'],axis=1,inplace=True)

In [13]:
df.shape

(1000, 40)

In [14]:
df.head()

,months_as_customer,age,policy_number,policy_bind_date,policy_state,policy_csl,policy_deductable,policy_annual_premium,umbrella_limit,insured_zip,insured_sex,insured_education_level,insured_occupation,insured_hobbies,insured_relationship,capital-gains,capital-loss,incident_date,incident_type,collision_type,incident_severity,authorities_contacted,incident_state,incident_city,incident_location,incident_hour_of_the_day,number_of_vehicles_involved,property_damage,bodily_injuries,witnesses,police_report_available,total_claim_amount,injury_claim,property_claim,vehicle_claim,auto_make,auto_model,auto_year,incident_date2policy_bind_date,target
0,328,48,521585,2014-10-17,OH,250/500,1000,1406.91,0,466132,MALE,MD,craft-repair,sleeping,husband,53300,0,2015-01-25,Single Vehicle Collision,Side Collision,Major Damage,Police,SC,Columbus,9935 4th Drive,5,1,YES,1,2,YES,71610,6510,13020,52080,Saab,92x,2004,100,1
1,228,42,342868,2006-06-27,IN,250/500,2000,1197.22,5000000,468176,MALE,MD,machine-op-inspct,reading,other-relative,0,0,2015-01-21,Vehicle Theft,?,Minor Damage,Police,VA,Riverwood,6608 MLK Hwy,8,1,?,0,0,?,5070,780,780,3510,Mercedes,E400,2007,3130,1
2,134,29,687698,2000-09-06,OH,100/300,2000,1413.14,5000000,430632,FEMALE,PhD,sales,board-games,own-child,35100,0,2015-02-22,Multi-vehicle Collision,Rear Collision,Minor Damage,Police,NY,Columbus,7121 Francis Lane,7,3,NO,2,3,NO,34650,7700,3850,23100,Dodge,RAM,2007,5282,0
3,256,41,227811,1990-05-25,IL,250/500,2000,1415.74,6000000,608117,FEMALE,PhD,armed-forces,board-games,unmarried,48900,-62400,2015-01-10,Single Vehicle Collision,Front Collision,Major Damage,Police,OH,Arlington,6956 Maple Drive,5,1,?,1,2,NO,63400,6340,6340,50720,Chevrolet,Tahoe,2014,8996,1
4,228,44,367455,2014-06-06,IL,500/1000,1000,1583.91,6000000,610706,MALE,Associate,sales,board-games,unmarried,66000,-46000,2015-02-17,Vehicle Theft,?,Minor Damage,None,NY,Arlington,3041 3rd Ave,20,1,NO,0,1,NO,6500,1300,650,4550,Accura,RSX,2009,256,0


Model

In [15]:
# Init H20
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "12.0.1" 2019-04-16; Java(TM) SE Runtime Environment (build 12.0.1+12); Java HotSpot(TM) 64-Bit Server VM (build 12.0.1+12, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/3p/_2mkzm0j70jc8cb2q69z4bx8fs8_kb/T/tmplbnmu3l3
  JVM stdout: /var/folders/3p/_2mkzm0j70jc8cb2q69z4bx8fs8_kb/T/tmplbnmu3l3/h2o_rohan_kothapalli_started_from_python.out
  JVM stderr: /var/folders/3p/_2mkzm0j70jc8cb2q69z4bx8fs8_kb/T/tmplbnmu3l3/h2o_rohan_kothapalli_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,14 days and 30 minutes
H2O_cluster_name:,H2O_from_python_rohan_kothapalli_hc9tdf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"accepting new members, healthy"


In [16]:
# train text split
seed = 56

hf = h2o.H2OFrame(df)
hf['target'] =hf['target'].asfactor()

train,test,valid = hf.split_frame(ratios=[0.7, 0.15])

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [17]:
hf.shape

(1000, 40)

In [18]:

target_label = 'target'
features_list = [x for x in hf.columns if x != target_label]

# for feature in features_list:
#     if "insured_zip" in feature:
        

In [19]:
# Train model
aml = H2OAutoML(max_runtime_secs=60 * 30, include_algos=["XGBoost"], seed=seed)

aml.train(x=features_list, y=target_label, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [20]:
h2o_result = get_leaderboard(aml, extra_columns='ALL').as_data_frame().loc[0]

model_name = h2o_result['model_id']
best_model = h2o.get_model(model_name)

preds = best_model.predict(test[:-1])
global_predict_res = preds.as_data_frame()
run_time = h2o_result['training_time_ms']

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [21]:
preds

predict,p0,p1
0,0.982144,0.0178564
0,0.982963,0.0170366
0,0.690042,0.309958
0,0.929159,0.0708408
0,0.945228,0.0547715
0,0.6773,0.3227
0,0.96754,0.0324601
0,0.983765,0.0162349
0,0.778969,0.221031
1,0.140655,0.859345


In [22]:
h2o_result.auc

0.8628513356562137

In [23]:
aml.download_mojo(path="auto_test.zip")

'/Users/rohan.kothapalli/Downloads/insurance/auto_test.zip'